In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

df= pd.read_csv('C://Users//User//Desktop//MSc Westminster//Dissertation//DataSets//Heart_Attack_Prediction.csv')

In [2]:
columns=df.columns

for column in columns:
    if df[column].dtype=="int32":
        df[column]=df[column].astype("int16")
    elif df[column].dtype=="float64":
        df[column]=df[column].astype("float16")
    elif df[column].dtype=="object":
        df[column]=df[column].astype("category")

In [3]:
df['Sex']= df['Sex'].map({'Female': 0, 'Male': 1})
df['Sex']= pd.to_numeric(df['Sex'])

df['Diet']= df['Diet'].map({'Healthy': 0, 'Average': 1, 'Unhealthy':2})
df['Diet']= pd.to_numeric(df['Diet'])

df[['HBP', 'LBP']]= df['Blood Pressure'].str.split('/', expand= True)
df['HBP']= pd.to_numeric(df['HBP'])
df['LBP']= pd.to_numeric(df['LBP'])

df['Diabetes'] = df['Diabetes'].map({0: 1, 1: 0})

df['Exercise Hours Per Week']= round(df['Exercise Hours Per Week'], 0)

df['Sedentary Hours Per Day']= round(df['Sedentary Hours Per Day'], 0)

df['Income']= round(df['Income'], 0)

df['BMI']= round(df['BMI'], 0)

df = df.drop(columns=['Patient ID', 'Blood Pressure', 'Country', 'Continent', 'Hemisphere'])

In [4]:
X= df.drop(['Heart Attack Risk'], axis= 1)
y= df['Heart Attack Risk']

In [5]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test= train_test_split(X, y, test_size= 0.2, random_state= 15, stratify= y)

from sklearn.preprocessing import StandardScaler
ss= StandardScaler()
X_train= ss.fit_transform(X_train)
X_test= ss.transform(X_test)

In [6]:
from imblearn.combine import SMOTETomek

smt= SMOTETomek(random_state= 15)
X_sm_tl, y_sm_tl= smt.fit_resample(X_train, y_train)

In [7]:
from sklearn.feature_selection import RFE

--- Tuned - DecisionTreeClassifier ---

In [8]:
from sklearn.metrics import confusion_matrix, classification_report, precision_score, roc_auc_score, accuracy_score
from sklearn.tree import DecisionTreeClassifier
dt= DecisionTreeClassifier(random_state= 15)

from sklearn.model_selection import GridSearchCV
param_grid= {
             'max_depth': [None, 10, 20, 30, 40, 50],
             'min_samples_split': [2, 10, 20],
             'min_samples_leaf': [1, 5, 10],
             'max_features': [None, 'sqrt', 'log2'],
             'criterion': ['gini', 'entropy']
            }

gs_dt= GridSearchCV(estimator= dt, param_grid= param_grid, cv= 5, scoring= 'precision')
gs_dt.fit(X_sm_tl, y_sm_tl)

tuned_dt= gs_dt.best_estimator_

selector= RFE(estimator= tuned_dt, n_features_to_select=10)
selector= selector.fit(X_sm_tl, y_sm_tl)
selected_features= selector.support_

X_sm_tl_selected= X_sm_tl[:, selected_features]
X_sm_tl_test_selected= X_test[:, selected_features]
tuned_dt.fit(X_sm_tl_selected, y_sm_tl)

y_pred_tuned_dt= tuned_dt.predict(X_sm_tl_test_selected)
y_pred_prob_tuned_dt= tuned_dt.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_dt, labels= tuned_dt.classes_)
print(cm)
print(classification_report(y_test, y_pred_tuned_dt, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_dt, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_dt))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_dt))

[[1026   99]
 [ 565   63]]
              precision    recall  f1-score   support

           0       0.64      0.91      0.76      1125
           1       0.39      0.10      0.16       628

    accuracy                           0.62      1753
   macro avg       0.52      0.51      0.46      1753
weighted avg       0.55      0.62      0.54      1753

Precision: 0.3888888888888889
Accuracy: 0.6212207644038791
AUC: 0.5058414720452937


--- Tuned Decision Tree 1 - RandomizedSearchCV ---

In [9]:
from sklearn.metrics import confusion_matrix, classification_report, precision_score, roc_auc_score, accuracy_score, make_scorer
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import RandomizedSearchCV
dt= DecisionTreeClassifier(random_state= 15)

param_dist = {
    'max_depth': [None, 10, 20, 30, 40, 50],
    'min_samples_split': [2, 5, 10, 20, 50],
    'min_samples_leaf': [1, 2, 5, 10, 20],
    'max_features': [None, 'sqrt', 'log2'],
    'criterion': ['gini', 'entropy'],
    'class_weight': [None, 'balanced']
}

scorer= make_scorer(precision_score, average= 'binary')

rs_dt= RandomizedSearchCV(estimator=dt, param_distributions=param_dist, n_iter= 100, cv= 5, scoring= scorer, random_state= 15)
rs_dt.fit(X_sm_tl, y_sm_tl)

tuned_dt1= rs_dt.best_estimator_

selector= RFE(estimator= tuned_dt1, n_features_to_select= 10)
selector= selector.fit(X_sm_tl, y_sm_tl)
selected_features= selector.support_

X_sm_tl_selected= X_sm_tl[:, selected_features]
X_sm_tl_test_selected= X_test[:, selected_features]
tuned_dt1.fit(X_sm_tl_selected, y_sm_tl)

y_pred_tuned_dt1= tuned_dt1.predict(X_sm_tl_test_selected)
y_pred_prob_tuned_dt1= tuned_dt1.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_dt1, labels= tuned_dt1.classes_)
print(cm)
print(classification_report(y_test, y_pred_tuned_dt1, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_dt1, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_dt1))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_dt1))

[[984 141]
 [546  82]]
              precision    recall  f1-score   support

           0       0.64      0.87      0.74      1125
           1       0.37      0.13      0.19       628

    accuracy                           0.61      1753
   macro avg       0.51      0.50      0.47      1753
weighted avg       0.54      0.61      0.54      1753

Precision: 0.36771300448430494
Accuracy: 0.6081003993154592
AUC: 0.5131408351026185


--- Tuned Decision Tree 2 - RandomizedSearchCV ---

In [10]:
from sklearn.metrics import confusion_matrix, classification_report, precision_score, roc_auc_score, accuracy_score, make_scorer
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import RandomizedSearchCV
dt= DecisionTreeClassifier(random_state= 15)

param_dist= {
    'max_depth': [None, 10, 20, 30, 40, 50, 60, 70, 80],
    'min_samples_split': [2, 5, 10, 20, 50, 100],
    'min_samples_leaf': [1, 2, 5, 10, 20, 50],
    'max_features': [None, 'sqrt', 'log2'],
    'criterion': ['gini', 'entropy'],
    'class_weight': [None, 'balanced']
}

scorer= make_scorer(precision_score, average= 'binary')

rs_dt= RandomizedSearchCV(estimator=dt, param_distributions=param_dist, n_iter= 200, cv= 5, scoring= scorer, random_state= 15)
rs_dt.fit(X_sm_tl, y_sm_tl)

tuned_dt2= rs_dt.best_estimator_

selector= RFE(estimator= tuned_dt2, n_features_to_select= 10)
selector= selector.fit(X_sm_tl, y_sm_tl)
selected_features= selector.support_

X_sm_tl_selected= X_sm_tl[:, selected_features]
X_sm_tl_test_selected= X_test[:, selected_features]
tuned_dt2.fit(X_sm_tl_selected, y_sm_tl)

y_pred_tuned_dt2= tuned_dt2.predict(X_sm_tl_test_selected)
y_pred_prob_tuned_dt2= tuned_dt2.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_dt2, labels= tuned_dt.classes_)
print(cm)
print(classification_report(y_test, y_pred_tuned_dt2, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_dt2, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_dt2))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_dt2))

[[1018  107]
 [ 572   56]]
              precision    recall  f1-score   support

           0       0.64      0.90      0.75      1125
           1       0.34      0.09      0.14       628

    accuracy                           0.61      1753
   macro avg       0.49      0.50      0.45      1753
weighted avg       0.53      0.61      0.53      1753

Precision: 0.34355828220858897
Accuracy: 0.6126640045636053
AUC: 0.5020976645435244


--- Tuned Random Forest ---

In [11]:
from sklearn.ensemble import RandomForestClassifier
rf= RandomForestClassifier(random_state= 15)

param_grid= {
             'n_estimators': [100, 200, 300],
             'max_depth': [None, 10, 20, 30],
             'min_samples_split': [2, 5, 10],
             'min_samples_leaf': [1, 2, 4],
            }

gs_tuned_rf= GridSearchCV(estimator= rf, param_grid= param_grid, cv= 5, scoring= 'precision', n_jobs= -1, verbose= 2)
gs_tuned_rf.fit(X_sm_tl, y_sm_tl)

tuned_rf= gs_tuned_rf.best_estimator_

selector= RFE(estimator= tuned_rf, n_features_to_select=10)
selector= selector.fit(X_sm_tl, y_sm_tl)
selected_features= selector.support_

X_sm_tl_selected= X_sm_tl[:, selected_features]
X_sm_tl_test_selected= X_test[:, selected_features]

tuned_rf.fit(X_sm_tl_selected, y_sm_tl)
y_pred_tuned_rf= tuned_rf.predict(X_sm_tl_test_selected)
y_pred_prob_tuned_rf= tuned_rf.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_rf, labels= tuned_rf.classes_)
print(cm)
print(classification_report(y_test, y_pred_tuned_rf, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_rf, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_rf))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_rf))

Fitting 5 folds for each of 108 candidates, totalling 540 fits
[[969 156]
 [530  98]]
              precision    recall  f1-score   support

           0       0.65      0.86      0.74      1125
           1       0.39      0.16      0.22       628

    accuracy                           0.61      1753
   macro avg       0.52      0.51      0.48      1753
weighted avg       0.55      0.61      0.55      1753

Precision: 0.3858267716535433
Accuracy: 0.6086708499714775
AUC: 0.5258917197452229


--- Tuned Random Forest 1 - RandomizedSearchCV ---

In [12]:
from sklearn.ensemble import RandomForestClassifier
rf= RandomForestClassifier(random_state= 15)

param_dist= {
    'n_estimators': [100, 200, 300, 500, 700],
    'max_depth': [None, 10, 20, 30, 40, 50],
    'min_samples_split': [2, 5, 10, 20],
    'min_samples_leaf': [1, 2, 4, 10],
    'max_features': ['auto', 'sqrt', 'log2'],
    'bootstrap': [True, False],
    'class_weight': [None, 'balanced']
}

scorer= make_scorer(precision_score, average= 'binary')
rs_tuned_rf= RandomizedSearchCV(estimator= rf, param_distributions= param_dist, n_iter= 100, cv= 5, scoring= scorer, random_state= 15, n_jobs= -1, verbose= 2)
rs_tuned_rf.fit(X_sm_tl, y_sm_tl)

tuned_rf1= rs_tuned_rf.best_estimator_

selector= RFE(estimator= tuned_rf1, n_features_to_select=10)
selector= selector.fit(X_sm_tl, y_sm_tl)
selected_features= selector.support_

X_sm_tl_selected= X_sm_tl[:, selected_features]
X_sm_tl_test_selected= X_test[:, selected_features]

tuned_rf1.fit(X_sm_tl_selected, y_sm_tl)
y_pred_tuned_rf1= tuned_rf1.predict(X_sm_tl_test_selected)
y_pred_prob_tuned_rf1= tuned_rf1.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_rf1, labels= tuned_rf1.classes_)
print(cm)
print(classification_report(y_test, y_pred_tuned_rf1, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_rf1, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_rf1))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_rf1))

Fitting 5 folds for each of 100 candidates, totalling 500 fits
[[965 160]
 [520 108]]
              precision    recall  f1-score   support

           0       0.65      0.86      0.74      1125
           1       0.40      0.17      0.24       628

    accuracy                           0.61      1753
   macro avg       0.53      0.51      0.49      1753
weighted avg       0.56      0.61      0.56      1753

Precision: 0.40298507462686567
Accuracy: 0.612093553907587
AUC: 0.5316666666666666


--- Tuned Random Forest 2 - RandomizedSearchCV ---

In [13]:
from sklearn.ensemble import RandomForestClassifier
rf= RandomForestClassifier(random_state= 15)

param_dist = {
    'n_estimators': [100, 200, 300, 500, 700, 1000],
    'max_depth': [None, 10, 20, 30, 40, 50, 60],
    'min_samples_split': [2, 5, 10, 20, 50],
    'min_samples_leaf': [1, 2, 4, 10, 20],
    'max_features': ['auto', 'sqrt', 'log2'],
    'bootstrap': [True, False],
    'class_weight': [{0: 1, 1: x} for x in range(1, 10)]
}

scorer= make_scorer(precision_score, average= 'binary')
rs_tuned_rf= RandomizedSearchCV(estimator= rf, param_distributions= param_dist, n_iter= 100, cv= 5, scoring= scorer, random_state= 15, n_jobs= -1, verbose= 2)
rs_tuned_rf.fit(X_sm_tl, y_sm_tl)

tuned_rf2= rs_tuned_rf.best_estimator_

selector= RFE(estimator= tuned_rf2, n_features_to_select=10)
selector= selector.fit(X_sm_tl, y_sm_tl)
selected_features= selector.support_

X_sm_tl_selected= X_sm_tl[:, selected_features]
X_sm_tl_test_selected= X_test[:, selected_features]

tuned_rf2.fit(X_sm_tl_selected, y_sm_tl)
y_pred_tuned_rf2= tuned_rf2.predict(X_sm_tl_test_selected)
y_pred_prob_tuned_rf2= tuned_rf2.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_rf2, labels= tuned_rf2.classes_)
print(cm)
print(classification_report(y_test, y_pred_tuned_rf2, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_rf2, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_rf2))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_rf2))

Fitting 5 folds for each of 100 candidates, totalling 500 fits
[[913 212]
 [512 116]]
              precision    recall  f1-score   support

           0       0.64      0.81      0.72      1125
           1       0.35      0.18      0.24       628

    accuracy                           0.59      1753
   macro avg       0.50      0.50      0.48      1753
weighted avg       0.54      0.59      0.55      1753

Precision: 0.35365853658536583
Accuracy: 0.5869937250427838
AUC: 0.5016794055201699


--- Tuned AdaBoost ---

In [14]:
from sklearn.ensemble import AdaBoostClassifier
ada= AdaBoostClassifier(random_state= 15)

param_grid= {'n_estimators': [50, 100, 200]}

gs_ada= GridSearchCV(estimator= ada, param_grid= param_grid, cv= 5, scoring= 'precision', n_jobs= -1, verbose= 2)
gs_ada.fit(X_sm_tl, y_sm_tl)

tuned_ada= gs_ada.best_estimator_

selector= RFE(estimator= tuned_ada, n_features_to_select=10)
selector= selector.fit(X_sm_tl, y_sm_tl)
selected_features= selector.support_

X_sm_tl_selected= X_sm_tl[:, selected_features]
X_sm_tl_test_selected= X_test[:, selected_features]

tuned_ada.fit(X_sm_tl_selected, y_sm_tl)
y_pred_tuned_ada= tuned_ada.predict(X_sm_tl_test_selected)
y_pred_prob_tuned_ada= tuned_ada.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_ada, labels= tuned_ada.classes_)
print(cm)
print(classification_report(y_test, y_pred_tuned_ada, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_ada, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_ada))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_ada))

Fitting 5 folds for each of 3 candidates, totalling 15 fits
[[1110   15]
 [ 618   10]]
              precision    recall  f1-score   support

           0       0.64      0.99      0.78      1125
           1       0.40      0.02      0.03       628

    accuracy                           0.64      1753
   macro avg       0.52      0.50      0.40      1753
weighted avg       0.56      0.64      0.51      1753

Precision: 0.4
Accuracy: 0.638904734740445
AUC: 0.4938542108987969


--- Tuned AdaBoost 1 - RandomizedSearchCV ---

In [15]:
from sklearn.ensemble import AdaBoostClassifier
ada= AdaBoostClassifier(random_state= 15)

param_dist= {
    'n_estimators': [50, 100, 200, 300, 500],
    'learning_rate': [0.001, 0.01, 0.1, 0.5, 1],
    'base_estimator': [DecisionTreeClassifier(max_depth=1),
                       DecisionTreeClassifier(max_depth=2),
                       DecisionTreeClassifier(max_depth=3),
                       DecisionTreeClassifier(max_depth=5)]
}

scorer= make_scorer(precision_score, average= 'binary')

rs_tuned_ada = RandomizedSearchCV(estimator= ada, param_distributions= param_dist, n_iter= 100, cv= 5, scoring= scorer, random_state= 15, n_jobs= -1, verbose= 2)
rs_tuned_ada.fit(X_sm_tl, y_sm_tl)

tuned_ada1= rs_tuned_ada.best_estimator_

selector= RFE(estimator= tuned_ada1, n_features_to_select=10)
selector= selector.fit(X_sm_tl, y_sm_tl)
selected_features= selector.support_

X_sm_tl_selected= X_sm_tl[:, selected_features]
X_sm_tl_test_selected= X_test[:, selected_features]

tuned_ada1.fit(X_sm_tl_selected, y_sm_tl)
y_pred_tuned_ada1= tuned_ada1.predict(X_sm_tl_test_selected)
y_pred_prob_tuned_ada1= tuned_ada1.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_ada1, labels= tuned_ada1.classes_)
print(cm)
print(classification_report(y_test, y_pred_tuned_ada1, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_ada1, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_ada1))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_ada1))

Fitting 5 folds for each of 100 candidates, totalling 500 fits


C:\Users\User\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\User\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\User\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\User\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\User\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\User\anaconda3\lib\site-package

[[1120    5]
 [ 626    2]]
              precision    recall  f1-score   support

           0       0.64      1.00      0.78      1125
           1       0.29      0.00      0.01       628

    accuracy                           0.64      1753
   macro avg       0.46      0.50      0.39      1753
weighted avg       0.51      0.64      0.50      1753

Precision: 0.2857142857142857
Accuracy: 0.6400456360524814
AUC: 0.5094564755838641


--- Tuned AdaBoost 2 - RandomizedSearchCV ---

In [16]:
from sklearn.ensemble import AdaBoostClassifier
ada= AdaBoostClassifier(random_state= 15)

param_dist= {
    'n_estimators': [50, 100, 200, 300, 500, 800, 1000],
    'learning_rate': [0.001, 0.01, 0.1, 0.5, 1],
    'base_estimator': [
        DecisionTreeClassifier(max_depth=1),
        DecisionTreeClassifier(max_depth=2),
        DecisionTreeClassifier(max_depth=3),
        DecisionTreeClassifier(max_depth=5),
        DecisionTreeClassifier(max_depth=10)
    ],
    'algorithm': ['SAMME', 'SAMME.R']
}

scorer= make_scorer(precision_score, average= 'binary', zero_division= 0)

rs_tuned_ada= RandomizedSearchCV(estimator= ada, param_distributions= param_dist, n_iter= 100, cv= 5, scoring= scorer, random_state= 15, n_jobs= -1, verbose= 2)
rs_tuned_ada.fit(X_sm_tl, y_sm_tl)

tuned_ada2= rs_tuned_ada.best_estimator_

selector= RFE(estimator= tuned_ada2, n_features_to_select=10)
selector= selector.fit(X_sm_tl, y_sm_tl)
selected_features= selector.support_

X_sm_tl_selected= X_sm_tl[:, selected_features]
X_sm_tl_test_selected= X_test[:, selected_features]

tuned_ada2.fit(X_sm_tl_selected, y_sm_tl)
y_pred_tuned_ada2= tuned_ada2.predict(X_sm_tl_test_selected)
y_pred_prob_tuned_ada2= tuned_ada2.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_ada2, labels= tuned_ada2.classes_)
print(cm)
print(classification_report(y_test, y_pred_tuned_ada2, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_ada2, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_ada2))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_ada2))

Fitting 5 folds for each of 100 candidates, totalling 500 fits


C:\Users\User\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\User\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\User\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\User\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\User\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\User\anaconda3\lib\site-package

[[920 205]
 [500 128]]
              precision    recall  f1-score   support

           0       0.65      0.82      0.72      1125
           1       0.38      0.20      0.27       628

    accuracy                           0.60      1753
   macro avg       0.52      0.51      0.49      1753
weighted avg       0.55      0.60      0.56      1753

Precision: 0.3843843843843844
Accuracy: 0.5978322875071307
AUC: 0.5059051663128096


--- Tuned AdaBoost 3 - RandomizedSearchCV ---

In [17]:
from sklearn.ensemble import AdaBoostClassifier
ada= AdaBoostClassifier(random_state= 15)

param_dist = {
    'n_estimators': [50, 100, 200, 300, 500, 800, 1000],
    'learning_rate': [0.001, 0.01, 0.1, 0.5, 1],
    'base_estimator': [
        DecisionTreeClassifier(max_depth=1, class_weight={0:1, 1:10}),
        DecisionTreeClassifier(max_depth=2, class_weight={0:1, 1:10}),
        DecisionTreeClassifier(max_depth=3, class_weight={0:1, 1:10}),
        DecisionTreeClassifier(max_depth=5, class_weight={0:1, 1:10}),
        DecisionTreeClassifier(max_depth=10, class_weight={0:1, 1:10})
    ],
    'algorithm': ['SAMME', 'SAMME.R']
}

scorer= make_scorer(precision_score, average= 'binary', zero_division= 0)

rs_tuned_ada= RandomizedSearchCV(estimator= ada, param_distributions= param_dist, n_iter= 100, cv= 5, scoring= scorer, random_state= 15, n_jobs= -1, verbose= 2)
rs_tuned_ada.fit(X_sm_tl, y_sm_tl)

tuned_ada3= rs_tuned_ada.best_estimator_

selector= RFE(estimator= tuned_ada3, n_features_to_select=10)
selector= selector.fit(X_sm_tl, y_sm_tl)
selected_features= selector.support_

X_sm_tl_selected= X_sm_tl[:, selected_features]
X_sm_tl_test_selected= X_test[:, selected_features]

tuned_ada3.fit(X_sm_tl_selected, y_sm_tl)
y_pred_tuned_ada3= tuned_ada3.predict(X_sm_tl_test_selected)
y_pred_prob_tuned_ada3= tuned_ada3.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_ada3, labels= tuned_ada3.classes_)
print(cm)
print(classification_report(y_test, y_pred_tuned_ada3, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_ada3, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_ada3))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_ada3))

Fitting 5 folds for each of 100 candidates, totalling 500 fits


C:\Users\User\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:378: FitFailedWarning: 
46 fits failed out of a total of 500.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
46 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\User\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\User\anaconda3\lib\site-packages\sklearn\ensemble\_weight_boosting.py", line 162, in fit
    sample_weight, estimator_weight, estimator_error = self._boost(
  File "C:\Users\User\anaconda3\lib\site-packages\sklearn\ensemble\_weight_boosting.py", line 572, in _boost
    return self._boost_discr

[[881 244]
 [489 139]]
              precision    recall  f1-score   support

           0       0.64      0.78      0.71      1125
           1       0.36      0.22      0.27       628

    accuracy                           0.58      1753
   macro avg       0.50      0.50      0.49      1753
weighted avg       0.54      0.58      0.55      1753

Precision: 0.3629242819843342
Accuracy: 0.5818596691386195
AUC: 0.4900169851380043


--- GradientBoosting ---

In [11]:
from sklearn.metrics import confusion_matrix, classification_report, precision_score, roc_auc_score, accuracy_score, make_scorer
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import GradientBoostingClassifier
grb= GradientBoostingClassifier(random_state=15)

selector= RFE(estimator= grb, n_features_to_select=10)
selector= selector.fit(X_sm_tl, y_sm_tl)
selected_features= selector.support_

X_sm_tl_selected= X_sm_tl[:, selected_features]
X_sm_tl_test_selected= X_test[:, selected_features]


grb.fit(X_sm_tl_selected, y_sm_tl)
y_pred_grb= grb.predict(X_sm_tl_test_selected)
y_pred_prob_grb= grb.predict_proba(X_sm_tl_test_selected)[:,1]


cm= confusion_matrix(y_test, y_pred_grb, labels= grb.classes_)
print(cm)
print(classification_report(y_test, y_pred_grb, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_grb, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_grb))
print('AUC:', roc_auc_score(y_test, y_pred_prob_grb))

X_sm_tl_df= pd.DataFrame(X_sm_tl, columns= X.columns)
selected_feature_names= X_sm_tl_df.columns[selected_features]
print("Selected Features:", selected_feature_names)

[[1095   30]
 [ 607   21]]
              precision    recall  f1-score   support

           0       0.64      0.97      0.77      1125
           1       0.41      0.03      0.06       628

    accuracy                           0.64      1753
   macro avg       0.53      0.50      0.42      1753
weighted avg       0.56      0.64      0.52      1753

Precision: 0.4117647058823529
Accuracy: 0.6366229321163719
AUC: 0.5132448690728946
Selected Features: Index(['Cholesterol', 'Obesity', 'Alcohol Consumption', 'Diet', 'Stress Level',
       'Sedentary Hours Per Day', 'Income', 'BMI',
       'Physical Activity Days Per Week', 'Sleep Hours Per Day'],
      dtype='object')


--- Tuned GradientBoosting ---

In [19]:
param_grid= {'learning_rate': [0.01, 0.1, 0.2]}

gs_grb= GridSearchCV(estimator= grb, param_grid= param_grid, cv= 5, scoring= 'precision', n_jobs= -1, verbose= 2)
gs_grb.fit(X_sm_tl, y_sm_tl)

tuned_grb= gs_grb.best_estimator_

selector= RFE(estimator= tuned_grb, n_features_to_select=10)
selector= selector.fit(X_sm_tl, y_sm_tl)
selected_features= selector.support_

X_sm_tl_selected= X_sm_tl[:, selected_features]
X_sm_tl_test_selected= X_test[:, selected_features]

tuned_grb.fit(X_sm_tl_selected, y_sm_tl)
y_pred_tuned_grb= tuned_grb.predict(X_sm_tl_test_selected)
y_pred_prob_tuned_grb= tuned_grb.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_grb, labels= tuned_grb.classes_)
print(cm)
print(classification_report(y_test, y_pred_tuned_grb, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_grb, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_grb))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_grb))

Fitting 5 folds for each of 3 candidates, totalling 15 fits
[[1065   60]
 [ 599   29]]
              precision    recall  f1-score   support

           0       0.64      0.95      0.76      1125
           1       0.33      0.05      0.08       628

    accuracy                           0.62      1753
   macro avg       0.48      0.50      0.42      1753
weighted avg       0.53      0.62      0.52      1753

Precision: 0.3258426966292135
Accuracy: 0.6240730176839704
AUC: 0.5161910828025478


--- Tuned GradientBoosting 1 - RandomizedSearchCV ---

In [12]:


param_dist= {
    'n_estimators': [100, 200, 300, 500, 800, 1000],
    'learning_rate': [0.001, 0.01, 0.1, 0.2, 0.5],
    'max_depth': [3, 5, 7, 10],
    'subsample': [0.8, 0.9, 1.0],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

scorer= make_scorer(precision_score, average= 'binary', zero_division =0)

rs_tuned_grb= RandomizedSearchCV(estimator= grb, param_distributions= param_dist, n_iter= 100, cv= 5, scoring= scorer, random_state= 15, n_jobs= -1, verbose= 2)
rs_tuned_grb.fit(X_sm_tl, y_sm_tl)

tuned_grb1= rs_tuned_grb.best_estimator_

selector= RFE(estimator= tuned_grb1, n_features_to_select=10)
selector= selector.fit(X_sm_tl, y_sm_tl)
selected_features= selector.support_

X_sm_tl_selected= X_sm_tl[:, selected_features]
X_sm_tl_test_selected= X_test[:, selected_features]

tuned_grb1.fit(X_sm_tl_selected, y_sm_tl)
y_pred_tuned_grb1= tuned_grb1.predict(X_sm_tl_test_selected)
y_pred_prob_tuned_grb1= tuned_grb1.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_grb1, labels= tuned_grb1.classes_)
print(cm)
print(classification_report(y_test, y_pred_tuned_grb1, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_grb1, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_grb1))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_grb1))

X_sm_tl_df= pd.DataFrame(X_sm_tl, columns= X.columns)
selected_feature_names= X_sm_tl_df.columns[selected_features]
print("Selected Features:", selected_feature_names)

Fitting 5 folds for each of 100 candidates, totalling 500 fits


KeyboardInterrupt: 

--- Tuned GradientBoosting 2 - RandomizedSearchCV ---

In [21]:
param_dist = {
    'n_estimators': [100, 200, 300, 500, 800, 1000],
    'learning_rate': [0.001, 0.01, 0.1, 0.2, 0.5],
    'max_depth': [3, 5, 7, 10],
    'subsample': [0.6, 0.8, 0.9, 1.0],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['auto', 'sqrt', 'log2', None],
    'n_iter_no_change': [None, 10, 20],
    'validation_fraction': [0.1, 0.2]
}

scorer= make_scorer(precision_score, average= 'binary', zero_division =0)

rs_tuned_grb= RandomizedSearchCV(estimator= grb, param_distributions= param_dist, n_iter= 100, cv= 5, scoring= scorer, random_state= 15, n_jobs= -1, verbose= 2)
rs_tuned_grb.fit(X_sm_tl, y_sm_tl)

tuned_grb2= rs_tuned_grb.best_estimator_

selector= RFE(estimator= tuned_grb2, n_features_to_select=10)
selector= selector.fit(X_sm_tl, y_sm_tl)
selected_features= selector.support_

X_sm_tl_selected= X_sm_tl[:, selected_features]
X_sm_tl_test_selected= X_test[:, selected_features]

tuned_grb2.fit(X_sm_tl_selected, y_sm_tl)
y_pred_tuned_grb2= tuned_grb2.predict(X_sm_tl_test_selected)
y_pred_prob_tuned_grb2= tuned_grb2.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_grb2, labels= tuned_grb2.classes_)
print(cm)
print(classification_report(y_test, y_pred_tuned_grb2, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_grb2, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_grb2))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_grb2))

Fitting 5 folds for each of 100 candidates, totalling 500 fits
[[910 215]
 [494 134]]
              precision    recall  f1-score   support

           0       0.65      0.81      0.72      1125
           1       0.38      0.21      0.27       628

    accuracy                           0.60      1753
   macro avg       0.52      0.51      0.50      1753
weighted avg       0.55      0.60      0.56      1753

Precision: 0.3839541547277937
Accuracy: 0.5955504848830576
AUC: 0.5226709129511677


--- Tuned CatBoost ---

In [22]:
from catboost import CatBoostClassifier
cat= CatBoostClassifier(random_state=15)

params_CatBoost= {
                  'depth': [3,5,10],
                  'learning_rate' : [0.01,0.1,1],
                  'iterations' : [5,10,50,100]
                 }

gs_cat= GridSearchCV(estimator= cat, param_grid= params_CatBoost, cv=5, scoring='precision', n_jobs=-1, verbose=2)
gs_cat.fit(X_sm_tl, y_sm_tl)

tuned_cat= gs_cat.best_estimator_

selector= RFE(estimator= tuned_cat, n_features_to_select=10)
selector= selector.fit(X_sm_tl, y_sm_tl)
selected_features= selector.support_

X_sm_tl_selected= X_sm_tl[:, selected_features]
X_sm_tl_test_selected= X_test[:, selected_features]

tuned_cat.fit(X_sm_tl_selected, y_sm_tl)
y_pred_tuned_cat= tuned_cat.predict(X_sm_tl_test_selected)
y_pred_prob_tuned_cat= tuned_cat.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_cat, labels= tuned_cat.classes_)
print(cm)
print(classification_report(y_test, y_pred_tuned_cat, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_cat, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_cat))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_cat))

Fitting 5 folds for each of 36 candidates, totalling 180 fits
0:	learn: 0.6907298	total: 209ms	remaining: 20.7s
1:	learn: 0.6894892	total: 271ms	remaining: 13.3s
2:	learn: 0.6885905	total: 327ms	remaining: 10.6s
3:	learn: 0.6869870	total: 380ms	remaining: 9.13s
4:	learn: 0.6853325	total: 434ms	remaining: 8.25s
5:	learn: 0.6834098	total: 494ms	remaining: 7.75s
6:	learn: 0.6814347	total: 546ms	remaining: 7.25s
7:	learn: 0.6801403	total: 602ms	remaining: 6.93s
8:	learn: 0.6788242	total: 657ms	remaining: 6.64s
9:	learn: 0.6776116	total: 709ms	remaining: 6.38s
10:	learn: 0.6759184	total: 763ms	remaining: 6.17s
11:	learn: 0.6746354	total: 817ms	remaining: 5.99s
12:	learn: 0.6723875	total: 869ms	remaining: 5.82s
13:	learn: 0.6711261	total: 922ms	remaining: 5.66s
14:	learn: 0.6698886	total: 972ms	remaining: 5.51s
15:	learn: 0.6686721	total: 1.02s	remaining: 5.38s
16:	learn: 0.6673800	total: 1.08s	remaining: 5.29s
17:	learn: 0.6663922	total: 1.14s	remaining: 5.18s
18:	learn: 0.6655287	total: 1.

63:	learn: 0.6181662	total: 3.2s	remaining: 1.8s
64:	learn: 0.6174932	total: 3.25s	remaining: 1.75s
65:	learn: 0.6167251	total: 3.29s	remaining: 1.7s
66:	learn: 0.6158252	total: 3.34s	remaining: 1.64s
67:	learn: 0.6150693	total: 3.38s	remaining: 1.59s
68:	learn: 0.6144151	total: 3.43s	remaining: 1.54s
69:	learn: 0.6135373	total: 3.48s	remaining: 1.49s
70:	learn: 0.6128884	total: 3.52s	remaining: 1.44s
71:	learn: 0.6120146	total: 3.57s	remaining: 1.39s
72:	learn: 0.6111377	total: 3.61s	remaining: 1.34s
73:	learn: 0.6103082	total: 3.67s	remaining: 1.29s
74:	learn: 0.6094837	total: 3.71s	remaining: 1.24s
75:	learn: 0.6086647	total: 3.76s	remaining: 1.19s
76:	learn: 0.6080368	total: 3.8s	remaining: 1.14s
77:	learn: 0.6069506	total: 3.85s	remaining: 1.09s
78:	learn: 0.6061362	total: 3.9s	remaining: 1.04s
79:	learn: 0.6054403	total: 3.95s	remaining: 987ms
80:	learn: 0.6047292	total: 3.99s	remaining: 937ms
81:	learn: 0.6041841	total: 4.04s	remaining: 887ms
82:	learn: 0.6035370	total: 4.09s	re

25:	learn: 0.6589805	total: 1.2s	remaining: 3.4s
26:	learn: 0.6571112	total: 1.24s	remaining: 3.37s
27:	learn: 0.6559695	total: 1.29s	remaining: 3.32s
28:	learn: 0.6546849	total: 1.33s	remaining: 3.27s
29:	learn: 0.6537753	total: 1.38s	remaining: 3.22s
30:	learn: 0.6516994	total: 1.43s	remaining: 3.17s
31:	learn: 0.6506954	total: 1.47s	remaining: 3.13s
32:	learn: 0.6499035	total: 1.52s	remaining: 3.09s
33:	learn: 0.6489765	total: 1.57s	remaining: 3.04s
34:	learn: 0.6477775	total: 1.61s	remaining: 3s
35:	learn: 0.6465322	total: 1.67s	remaining: 2.97s
36:	learn: 0.6456634	total: 1.73s	remaining: 2.94s
37:	learn: 0.6447218	total: 1.78s	remaining: 2.91s
38:	learn: 0.6433480	total: 1.84s	remaining: 2.88s
39:	learn: 0.6422342	total: 1.89s	remaining: 2.84s
40:	learn: 0.6412040	total: 1.96s	remaining: 2.81s
41:	learn: 0.6401346	total: 2.02s	remaining: 2.79s
42:	learn: 0.6391221	total: 2.08s	remaining: 2.76s
43:	learn: 0.6378668	total: 2.15s	remaining: 2.74s
44:	learn: 0.6372932	total: 2.21s	re

89:	learn: 0.6486588	total: 4.52s	remaining: 503ms
90:	learn: 0.6482488	total: 4.57s	remaining: 452ms
91:	learn: 0.6476485	total: 4.61s	remaining: 401ms
92:	learn: 0.6472625	total: 4.66s	remaining: 351ms
93:	learn: 0.6468876	total: 4.71s	remaining: 301ms
94:	learn: 0.6465335	total: 4.76s	remaining: 250ms
95:	learn: 0.6461853	total: 4.81s	remaining: 200ms
96:	learn: 0.6457631	total: 4.85s	remaining: 150ms
97:	learn: 0.6451705	total: 4.89s	remaining: 99.9ms
98:	learn: 0.6448098	total: 4.93s	remaining: 49.9ms
99:	learn: 0.6444781	total: 4.98s	remaining: 0us
0:	learn: 0.6925873	total: 40.3ms	remaining: 3.99s
1:	learn: 0.6919838	total: 82.2ms	remaining: 4.03s
2:	learn: 0.6913460	total: 127ms	remaining: 4.09s
3:	learn: 0.6909759	total: 165ms	remaining: 3.95s
4:	learn: 0.6902458	total: 206ms	remaining: 3.92s
5:	learn: 0.6898193	total: 242ms	remaining: 3.8s
6:	learn: 0.6890084	total: 282ms	remaining: 3.75s
7:	learn: 0.6885494	total: 323ms	remaining: 3.71s
8:	learn: 0.6880577	total: 361ms	remai

51:	learn: 0.6641542	total: 2.21s	remaining: 2.04s
52:	learn: 0.6637027	total: 2.25s	remaining: 1.99s
53:	learn: 0.6631290	total: 2.28s	remaining: 1.95s
54:	learn: 0.6627341	total: 2.32s	remaining: 1.9s
55:	learn: 0.6621924	total: 2.36s	remaining: 1.86s
56:	learn: 0.6613765	total: 2.4s	remaining: 1.81s
57:	learn: 0.6609518	total: 2.45s	remaining: 1.77s
58:	learn: 0.6605141	total: 2.49s	remaining: 1.73s
59:	learn: 0.6600328	total: 2.53s	remaining: 1.69s
60:	learn: 0.6596197	total: 2.57s	remaining: 1.64s
61:	learn: 0.6591431	total: 2.61s	remaining: 1.6s
62:	learn: 0.6587017	total: 2.65s	remaining: 1.55s
63:	learn: 0.6578819	total: 2.69s	remaining: 1.51s
64:	learn: 0.6573181	total: 2.73s	remaining: 1.47s
65:	learn: 0.6568552	total: 2.77s	remaining: 1.43s
66:	learn: 0.6562460	total: 2.81s	remaining: 1.38s
67:	learn: 0.6557926	total: 2.85s	remaining: 1.34s
68:	learn: 0.6554982	total: 2.89s	remaining: 1.3s
69:	learn: 0.6551732	total: 2.93s	remaining: 1.25s
70:	learn: 0.6547949	total: 2.97s	r

13:	learn: 0.6843837	total: 485ms	remaining: 2.98s
14:	learn: 0.6838723	total: 521ms	remaining: 2.95s
15:	learn: 0.6832850	total: 553ms	remaining: 2.9s
16:	learn: 0.6828072	total: 586ms	remaining: 2.86s
17:	learn: 0.6823617	total: 617ms	remaining: 2.81s
18:	learn: 0.6816198	total: 648ms	remaining: 2.76s
19:	learn: 0.6810888	total: 682ms	remaining: 2.73s
20:	learn: 0.6804157	total: 716ms	remaining: 2.69s
21:	learn: 0.6798714	total: 749ms	remaining: 2.65s
22:	learn: 0.6793183	total: 781ms	remaining: 2.62s
23:	learn: 0.6784973	total: 814ms	remaining: 2.58s
24:	learn: 0.6778718	total: 845ms	remaining: 2.54s
25:	learn: 0.6774808	total: 877ms	remaining: 2.5s
26:	learn: 0.6771248	total: 914ms	remaining: 2.47s
27:	learn: 0.6766841	total: 947ms	remaining: 2.43s
28:	learn: 0.6762325	total: 980ms	remaining: 2.4s
29:	learn: 0.6755210	total: 1.01s	remaining: 2.36s
30:	learn: 0.6748630	total: 1.05s	remaining: 2.33s
31:	learn: 0.6744163	total: 1.09s	remaining: 2.31s
32:	learn: 0.6736197	total: 1.12s	

77:	learn: 0.6518992	total: 2.52s	remaining: 712ms
78:	learn: 0.6515699	total: 2.56s	remaining: 679ms
79:	learn: 0.6507038	total: 2.59s	remaining: 647ms
80:	learn: 0.6503650	total: 2.62s	remaining: 614ms
81:	learn: 0.6499562	total: 2.65s	remaining: 581ms
82:	learn: 0.6496500	total: 2.68s	remaining: 548ms
83:	learn: 0.6493128	total: 2.72s	remaining: 518ms
84:	learn: 0.6489828	total: 2.75s	remaining: 485ms
85:	learn: 0.6485623	total: 2.78s	remaining: 453ms
86:	learn: 0.6480920	total: 2.81s	remaining: 420ms
87:	learn: 0.6475069	total: 2.84s	remaining: 388ms
88:	learn: 0.6471583	total: 2.88s	remaining: 355ms
89:	learn: 0.6468682	total: 2.9s	remaining: 323ms
90:	learn: 0.6463558	total: 2.94s	remaining: 291ms
91:	learn: 0.6459730	total: 2.98s	remaining: 259ms
92:	learn: 0.6455372	total: 3.01s	remaining: 227ms
93:	learn: 0.6449410	total: 3.04s	remaining: 194ms
94:	learn: 0.6444134	total: 3.07s	remaining: 162ms
95:	learn: 0.6439426	total: 3.1s	remaining: 129ms
96:	learn: 0.6434269	total: 3.13s

39:	learn: 0.6684585	total: 1.2s	remaining: 1.81s
40:	learn: 0.6679251	total: 1.23s	remaining: 1.77s
41:	learn: 0.6672812	total: 1.26s	remaining: 1.74s
42:	learn: 0.6664728	total: 1.29s	remaining: 1.71s
43:	learn: 0.6658100	total: 1.32s	remaining: 1.68s
44:	learn: 0.6654208	total: 1.35s	remaining: 1.65s
45:	learn: 0.6650355	total: 1.38s	remaining: 1.62s
46:	learn: 0.6645004	total: 1.41s	remaining: 1.59s
47:	learn: 0.6639607	total: 1.44s	remaining: 1.56s
48:	learn: 0.6634786	total: 1.47s	remaining: 1.53s
49:	learn: 0.6630442	total: 1.5s	remaining: 1.5s
50:	learn: 0.6624770	total: 1.52s	remaining: 1.46s
51:	learn: 0.6619907	total: 1.55s	remaining: 1.43s
52:	learn: 0.6614535	total: 1.58s	remaining: 1.4s
53:	learn: 0.6609449	total: 1.61s	remaining: 1.37s
54:	learn: 0.6605941	total: 1.64s	remaining: 1.34s
55:	learn: 0.6600557	total: 1.67s	remaining: 1.31s
56:	learn: 0.6595162	total: 1.7s	remaining: 1.28s
57:	learn: 0.6591500	total: 1.73s	remaining: 1.25s
58:	learn: 0.6587745	total: 1.76s	re

7:	learn: 0.6871345	total: 292ms	remaining: 3.36s
8:	learn: 0.6865085	total: 319ms	remaining: 3.22s
9:	learn: 0.6855816	total: 346ms	remaining: 3.12s
10:	learn: 0.6850148	total: 378ms	remaining: 3.06s
11:	learn: 0.6843668	total: 404ms	remaining: 2.96s
12:	learn: 0.6838457	total: 432ms	remaining: 2.89s
13:	learn: 0.6829845	total: 458ms	remaining: 2.81s
14:	learn: 0.6821658	total: 485ms	remaining: 2.75s
15:	learn: 0.6815477	total: 514ms	remaining: 2.7s
16:	learn: 0.6809532	total: 540ms	remaining: 2.64s
17:	learn: 0.6804239	total: 566ms	remaining: 2.58s
18:	learn: 0.6798962	total: 593ms	remaining: 2.53s
19:	learn: 0.6792169	total: 619ms	remaining: 2.48s
20:	learn: 0.6787732	total: 646ms	remaining: 2.43s
21:	learn: 0.6782033	total: 672ms	remaining: 2.38s
22:	learn: 0.6776585	total: 700ms	remaining: 2.34s
23:	learn: 0.6770710	total: 729ms	remaining: 2.31s
24:	learn: 0.6762137	total: 756ms	remaining: 2.27s
25:	learn: 0.6756065	total: 782ms	remaining: 2.23s
26:	learn: 0.6749738	total: 808ms	r

73:	learn: 0.6503498	total: 2.1s	remaining: 739ms
74:	learn: 0.6499362	total: 2.13s	remaining: 710ms
75:	learn: 0.6495062	total: 2.15s	remaining: 681ms
76:	learn: 0.6491541	total: 2.18s	remaining: 652ms
77:	learn: 0.6487676	total: 2.21s	remaining: 623ms
78:	learn: 0.6482679	total: 2.23s	remaining: 594ms
79:	learn: 0.6478564	total: 2.26s	remaining: 566ms
80:	learn: 0.6474515	total: 2.29s	remaining: 538ms
81:	learn: 0.6469713	total: 2.32s	remaining: 509ms
82:	learn: 0.6466601	total: 2.35s	remaining: 481ms
83:	learn: 0.6461782	total: 2.37s	remaining: 452ms
84:	learn: 0.6456950	total: 2.4s	remaining: 424ms
85:	learn: 0.6451924	total: 2.43s	remaining: 395ms
86:	learn: 0.6448427	total: 2.46s	remaining: 367ms
87:	learn: 0.6444556	total: 2.48s	remaining: 339ms
88:	learn: 0.6439184	total: 2.51s	remaining: 310ms
89:	learn: 0.6433866	total: 2.54s	remaining: 282ms
90:	learn: 0.6429136	total: 2.56s	remaining: 254ms
91:	learn: 0.6425614	total: 2.59s	remaining: 225ms
92:	learn: 0.6422461	total: 2.62s

--- Tuned CatBoost 1 ---

In [23]:
from catboost import CatBoostClassifier
cat= CatBoostClassifier(random_state=15, class_weights=[1, 3])

params_CatBoost = {
    'depth': [4, 6, 8],
    'learning_rate': [0.01, 0.1, 0.3],
    'iterations': [50, 100, 200],
    'l2_leaf_reg': [1, 3, 5]
}

gs_cat= GridSearchCV(estimator= cat, param_grid= params_CatBoost, cv=5, scoring='precision', n_jobs=-1, verbose=2)
gs_cat.fit(X_sm_tl, y_sm_tl)

tuned_cat1= gs_cat.best_estimator_

selector= RFE(estimator= tuned_cat1, n_features_to_select=10)
selector= selector.fit(X_sm_tl, y_sm_tl)
selected_features= selector.support_

X_sm_tl_selected= X_sm_tl[:, selected_features]
X_sm_tl_test_selected= X_test[:, selected_features]

tuned_cat1.fit(X_sm_tl_selected, y_sm_tl)
y_pred_tuned_cat1= tuned_cat1.predict(X_sm_tl_test_selected)
y_pred_prob_tuned_cat1= tuned_cat1.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_cat1, labels= tuned_cat1.classes_)
print(cm)
print(classification_report(y_test, y_pred_tuned_cat1, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_cat1, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_cat1))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_cat1))

Fitting 5 folds for each of 81 candidates, totalling 405 fits
0:	learn: 0.6118974	total: 16.6ms	remaining: 3.3s
1:	learn: 0.5743449	total: 36.9ms	remaining: 3.65s
2:	learn: 0.5389686	total: 51.4ms	remaining: 3.38s
3:	learn: 0.5246390	total: 69.1ms	remaining: 3.39s
4:	learn: 0.5116091	total: 84ms	remaining: 3.27s
5:	learn: 0.5018444	total: 98.6ms	remaining: 3.19s
6:	learn: 0.4937478	total: 112ms	remaining: 3.09s
7:	learn: 0.4806008	total: 126ms	remaining: 3.02s
8:	learn: 0.4719607	total: 139ms	remaining: 2.95s
9:	learn: 0.4627594	total: 153ms	remaining: 2.91s
10:	learn: 0.4568319	total: 166ms	remaining: 2.85s
11:	learn: 0.4489583	total: 180ms	remaining: 2.82s
12:	learn: 0.4432676	total: 193ms	remaining: 2.77s
13:	learn: 0.4361743	total: 207ms	remaining: 2.75s
14:	learn: 0.4296002	total: 223ms	remaining: 2.74s
15:	learn: 0.4221821	total: 241ms	remaining: 2.77s
16:	learn: 0.4159171	total: 255ms	remaining: 2.74s
17:	learn: 0.4091269	total: 269ms	remaining: 2.72s
18:	learn: 0.4040848	total:

170:	learn: 0.0634743	total: 2.69s	remaining: 456ms
171:	learn: 0.0629468	total: 2.7s	remaining: 440ms
172:	learn: 0.0621132	total: 2.72s	remaining: 425ms
173:	learn: 0.0615043	total: 2.74s	remaining: 409ms
174:	learn: 0.0608649	total: 2.75s	remaining: 393ms
175:	learn: 0.0600776	total: 2.77s	remaining: 378ms
176:	learn: 0.0593479	total: 2.79s	remaining: 363ms
177:	learn: 0.0588166	total: 2.81s	remaining: 347ms
178:	learn: 0.0582321	total: 2.82s	remaining: 331ms
179:	learn: 0.0576934	total: 2.84s	remaining: 316ms
180:	learn: 0.0571274	total: 2.86s	remaining: 300ms
181:	learn: 0.0564694	total: 2.88s	remaining: 284ms
182:	learn: 0.0560431	total: 2.89s	remaining: 269ms
183:	learn: 0.0556210	total: 2.91s	remaining: 253ms
184:	learn: 0.0550974	total: 2.92s	remaining: 237ms
185:	learn: 0.0540347	total: 2.94s	remaining: 221ms
186:	learn: 0.0537356	total: 2.95s	remaining: 205ms
187:	learn: 0.0528001	total: 2.97s	remaining: 189ms
188:	learn: 0.0523032	total: 2.98s	remaining: 174ms
189:	learn: 0

141:	learn: 0.0874786	total: 1.88s	remaining: 768ms
142:	learn: 0.0858440	total: 1.89s	remaining: 755ms
143:	learn: 0.0847797	total: 1.91s	remaining: 741ms
144:	learn: 0.0839573	total: 1.92s	remaining: 728ms
145:	learn: 0.0829446	total: 1.93s	remaining: 715ms
146:	learn: 0.0817898	total: 1.95s	remaining: 702ms
147:	learn: 0.0809572	total: 1.96s	remaining: 688ms
148:	learn: 0.0803105	total: 1.97s	remaining: 675ms
149:	learn: 0.0794969	total: 1.98s	remaining: 661ms
150:	learn: 0.0787010	total: 2s	remaining: 648ms
151:	learn: 0.0780814	total: 2.01s	remaining: 634ms
152:	learn: 0.0772791	total: 2.02s	remaining: 621ms
153:	learn: 0.0765818	total: 2.03s	remaining: 608ms
154:	learn: 0.0759177	total: 2.05s	remaining: 594ms
155:	learn: 0.0748120	total: 2.06s	remaining: 581ms
156:	learn: 0.0740778	total: 2.07s	remaining: 568ms
157:	learn: 0.0735494	total: 2.09s	remaining: 555ms
158:	learn: 0.0726029	total: 2.1s	remaining: 541ms
159:	learn: 0.0717151	total: 2.11s	remaining: 528ms
160:	learn: 0.07

102:	learn: 0.1382238	total: 1.42s	remaining: 1.34s
103:	learn: 0.1370780	total: 1.43s	remaining: 1.32s
104:	learn: 0.1356874	total: 1.45s	remaining: 1.31s
105:	learn: 0.1335788	total: 1.46s	remaining: 1.29s
106:	learn: 0.1318084	total: 1.47s	remaining: 1.28s
107:	learn: 0.1302054	total: 1.48s	remaining: 1.26s
108:	learn: 0.1287409	total: 1.5s	remaining: 1.25s
109:	learn: 0.1272368	total: 1.51s	remaining: 1.23s
110:	learn: 0.1251217	total: 1.52s	remaining: 1.22s
111:	learn: 0.1231127	total: 1.53s	remaining: 1.21s
112:	learn: 0.1213977	total: 1.55s	remaining: 1.19s
113:	learn: 0.1199440	total: 1.56s	remaining: 1.18s
114:	learn: 0.1185837	total: 1.57s	remaining: 1.16s
115:	learn: 0.1169931	total: 1.58s	remaining: 1.15s
116:	learn: 0.1157404	total: 1.59s	remaining: 1.13s
117:	learn: 0.1139078	total: 1.61s	remaining: 1.12s
118:	learn: 0.1116423	total: 1.62s	remaining: 1.1s
119:	learn: 0.1106358	total: 1.63s	remaining: 1.09s
120:	learn: 0.1099051	total: 1.65s	remaining: 1.07s
121:	learn: 0.

74:	learn: 0.1921653	total: 1.09s	remaining: 1.82s
75:	learn: 0.1902177	total: 1.1s	remaining: 1.8s
76:	learn: 0.1868715	total: 1.12s	remaining: 1.78s
77:	learn: 0.1844796	total: 1.13s	remaining: 1.77s
78:	learn: 0.1826383	total: 1.14s	remaining: 1.75s
79:	learn: 0.1807410	total: 1.16s	remaining: 1.73s
80:	learn: 0.1782895	total: 1.17s	remaining: 1.72s
81:	learn: 0.1766912	total: 1.18s	remaining: 1.7s
82:	learn: 0.1748497	total: 1.19s	remaining: 1.68s
83:	learn: 0.1727593	total: 1.21s	remaining: 1.67s
84:	learn: 0.1707607	total: 1.22s	remaining: 1.65s
85:	learn: 0.1686889	total: 1.23s	remaining: 1.64s
86:	learn: 0.1664949	total: 1.25s	remaining: 1.62s
87:	learn: 0.1645123	total: 1.26s	remaining: 1.6s
88:	learn: 0.1620105	total: 1.27s	remaining: 1.59s
89:	learn: 0.1596068	total: 1.29s	remaining: 1.57s
90:	learn: 0.1575859	total: 1.3s	remaining: 1.56s
91:	learn: 0.1552264	total: 1.31s	remaining: 1.54s
92:	learn: 0.1527077	total: 1.33s	remaining: 1.53s
93:	learn: 0.1507650	total: 1.34s	re

...

1:	learn: 0.5665025	total: 38.6ms	remaining: 3.82s
2:	learn: 0.5377332	total: 54.7ms	remaining: 3.59s
3:	learn: 0.5231121	total: 70.3ms	remaining: 3.44s
4:	learn: 0.5141821	total: 88.4ms	remaining: 3.45s
5:	learn: 0.5073600	total: 107ms	remaining: 3.45s
6:	learn: 0.5011297	total: 124ms	remaining: 3.42s
7:	learn: 0.4912044	total: 138ms	remaining: 3.32s
8:	learn: 0.4859883	total: 149ms	remaining: 3.17s
9:	learn: 0.4798068	total: 159ms	remaining: 3.02s
10:	learn: 0.4710042	total: 169ms	remaining: 2.9s
11:	learn: 0.4643474	total: 179ms	remaining: 2.81s
12:	learn: 0.4595528	total: 189ms	remaining: 2.71s
13:	learn: 0.4524660	total: 198ms	remaining: 2.63s
14:	learn: 0.4459275	total: 208ms	remaining: 2.56s
15:	learn: 0.4413444	total: 217ms	remaining: 2.49s
16:	learn: 0.4372372	total: 228ms	remaining: 2.46s
17:	learn: 0.4337227	total: 239ms	remaining: 2.42s
18:	learn: 0.4278696	total: 250ms	remaining: 2.38s
19:	learn: 0.4228403	total: 260ms	remaining: 2.34s
20:	learn: 0.4180451	total: 270ms	rem

166:	learn: 0.0891348	total: 1.73s	remaining: 342ms
167:	learn: 0.0878254	total: 1.74s	remaining: 332ms
168:	learn: 0.0871290	total: 1.75s	remaining: 321ms
169:	learn: 0.0862688	total: 1.76s	remaining: 311ms
170:	learn: 0.0856189	total: 1.77s	remaining: 300ms
171:	learn: 0.0850294	total: 1.78s	remaining: 290ms
172:	learn: 0.0842350	total: 1.79s	remaining: 279ms
173:	learn: 0.0833189	total: 1.8s	remaining: 269ms
174:	learn: 0.0824958	total: 1.81s	remaining: 259ms
175:	learn: 0.0817516	total: 1.82s	remaining: 248ms
176:	learn: 0.0808051	total: 1.83s	remaining: 238ms
177:	learn: 0.0800803	total: 1.84s	remaining: 227ms
178:	learn: 0.0792965	total: 1.85s	remaining: 217ms
179:	learn: 0.0785306	total: 1.86s	remaining: 207ms
180:	learn: 0.0776203	total: 1.87s	remaining: 197ms
181:	learn: 0.0771165	total: 1.88s	remaining: 186ms
182:	learn: 0.0762748	total: 1.89s	remaining: 176ms
183:	learn: 0.0755500	total: 1.9s	remaining: 165ms
184:	learn: 0.0750695	total: 1.91s	remaining: 155ms
185:	learn: 0.

134:	learn: 0.1308934	total: 1.4s	remaining: 673ms
135:	learn: 0.1294505	total: 1.41s	remaining: 662ms
136:	learn: 0.1283160	total: 1.42s	remaining: 651ms
137:	learn: 0.1273879	total: 1.43s	remaining: 640ms
138:	learn: 0.1261347	total: 1.43s	remaining: 630ms
139:	learn: 0.1245905	total: 1.44s	remaining: 619ms
140:	learn: 0.1233937	total: 1.45s	remaining: 608ms
141:	learn: 0.1222361	total: 1.46s	remaining: 597ms
142:	learn: 0.1215308	total: 1.47s	remaining: 587ms
143:	learn: 0.1207714	total: 1.48s	remaining: 576ms
144:	learn: 0.1197962	total: 1.49s	remaining: 565ms
145:	learn: 0.1187747	total: 1.5s	remaining: 555ms
146:	learn: 0.1175022	total: 1.51s	remaining: 544ms
147:	learn: 0.1165110	total: 1.52s	remaining: 534ms
148:	learn: 0.1153281	total: 1.53s	remaining: 523ms
149:	learn: 0.1143474	total: 1.54s	remaining: 512ms
150:	learn: 0.1134887	total: 1.55s	remaining: 502ms
151:	learn: 0.1121866	total: 1.55s	remaining: 491ms
152:	learn: 0.1113564	total: 1.56s	remaining: 481ms
153:	learn: 0.

110:	learn: 0.1747664	total: 1.24s	remaining: 994ms
111:	learn: 0.1729057	total: 1.26s	remaining: 989ms
112:	learn: 0.1717321	total: 1.27s	remaining: 977ms
113:	learn: 0.1699966	total: 1.28s	remaining: 964ms
114:	learn: 0.1687300	total: 1.29s	remaining: 952ms
115:	learn: 0.1674961	total: 1.3s	remaining: 940ms
116:	learn: 0.1660244	total: 1.31s	remaining: 928ms
117:	learn: 0.1644013	total: 1.32s	remaining: 915ms
118:	learn: 0.1631713	total: 1.33s	remaining: 903ms
119:	learn: 0.1617454	total: 1.34s	remaining: 891ms
120:	learn: 0.1609197	total: 1.35s	remaining: 879ms
121:	learn: 0.1599237	total: 1.36s	remaining: 867ms
122:	learn: 0.1589792	total: 1.37s	remaining: 856ms
123:	learn: 0.1578928	total: 1.38s	remaining: 844ms
124:	learn: 0.1557440	total: 1.39s	remaining: 832ms
125:	learn: 0.1542163	total: 1.4s	remaining: 820ms
126:	learn: 0.1522010	total: 1.41s	remaining: 809ms
127:	learn: 0.1507874	total: 1.42s	remaining: 797ms
128:	learn: 0.1497805	total: 1.43s	remaining: 785ms
129:	learn: 0.

74:	learn: 0.2400451	total: 811ms	remaining: 1.35s
75:	learn: 0.2370176	total: 821ms	remaining: 1.34s
76:	learn: 0.2351148	total: 831ms	remaining: 1.33s
77:	learn: 0.2330767	total: 840ms	remaining: 1.31s
78:	learn: 0.2310299	total: 850ms	remaining: 1.3s
79:	learn: 0.2291245	total: 859ms	remaining: 1.29s
80:	learn: 0.2272696	total: 869ms	remaining: 1.28s
81:	learn: 0.2255521	total: 879ms	remaining: 1.26s
82:	learn: 0.2231658	total: 889ms	remaining: 1.25s
83:	learn: 0.2216046	total: 899ms	remaining: 1.24s
84:	learn: 0.2198572	total: 909ms	remaining: 1.23s
85:	learn: 0.2177158	total: 918ms	remaining: 1.22s
86:	learn: 0.2153242	total: 928ms	remaining: 1.21s
87:	learn: 0.2142680	total: 937ms	remaining: 1.19s
88:	learn: 0.2129506	total: 947ms	remaining: 1.18s
89:	learn: 0.2114226	total: 957ms	remaining: 1.17s
90:	learn: 0.2092382	total: 966ms	remaining: 1.16s
91:	learn: 0.2076706	total: 975ms	remaining: 1.15s
92:	learn: 0.2056690	total: 985ms	remaining: 1.13s
93:	learn: 0.2038372	total: 995m

--- Tuned CatBoost 2 - RandomizedSearchCV ---

In [24]:
from catboost import CatBoostClassifier
cat= CatBoostClassifier(random_state=15, class_weights= [1, 4])

params_CatBoost = {
    'depth': [6, 8, 10],
    'learning_rate': [0.01, 0.05, 0.1],
    'iterations': [100, 200, 300],
    'l2_leaf_reg': [1, 3, 7],
    'bagging_temperature': [0.0, 0.5, 1.0],
    'random_strength': [1, 2, 5],
    'rsm': [0.8, 1.0]
}

gs_cat= GridSearchCV(estimator= cat, param_grid= params_CatBoost, cv=5, scoring='precision', n_jobs=-1, verbose=2)
gs_cat.fit(X_sm_tl, y_sm_tl)

tuned_cat2= gs_cat.best_estimator_

selector= RFE(estimator= tuned_cat2, n_features_to_select=10)
selector= selector.fit(X_sm_tl, y_sm_tl)
selected_features= selector.support_

X_sm_tl_selected= X_sm_tl[:, selected_features]
X_sm_tl_test_selected= X_test[:, selected_features]

tuned_cat2.fit(X_sm_tl_selected, y_sm_tl)
y_pred_tuned_cat2= tuned_cat2.predict(X_sm_tl_test_selected)
y_pred_prob_tuned_cat2= tuned_cat2.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_cat2, labels= tuned_cat2.classes_)
print(cm)
print(classification_report(y_test, y_pred_tuned_cat2, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_cat2, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_cat2))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_cat2))

Fitting 5 folds for each of 1458 candidates, totalling 7290 fits
0:	learn: 0.6493793	total: 64.8ms	remaining: 19.4s
1:	learn: 0.6129226	total: 139ms	remaining: 20.7s
2:	learn: 0.5844351	total: 207ms	remaining: 20.5s
3:	learn: 0.5602411	total: 270ms	remaining: 20s
4:	learn: 0.5385947	total: 345ms	remaining: 20.4s
5:	learn: 0.5203662	total: 415ms	remaining: 20.3s
6:	learn: 0.5073743	total: 480ms	remaining: 20.1s
7:	learn: 0.4960562	total: 542ms	remaining: 19.8s
8:	learn: 0.4832478	total: 610ms	remaining: 19.7s
9:	learn: 0.4749814	total: 677ms	remaining: 19.6s
10:	learn: 0.4675506	total: 760ms	remaining: 20s
11:	learn: 0.4606675	total: 829ms	remaining: 19.9s
12:	learn: 0.4552115	total: 898ms	remaining: 19.8s
13:	learn: 0.4491366	total: 967ms	remaining: 19.8s
14:	learn: 0.4445629	total: 1.03s	remaining: 19.6s
15:	learn: 0.4386319	total: 1.1s	remaining: 19.5s
16:	learn: 0.4350014	total: 1.17s	remaining: 19.5s
17:	learn: 0.4305175	total: 1.23s	remaining: 19.3s
18:	learn: 0.4251630	total: 1.2

162:	learn: 0.1157311	total: 10.2s	remaining: 8.59s
163:	learn: 0.1147850	total: 10.3s	remaining: 8.53s
164:	learn: 0.1139427	total: 10.3s	remaining: 8.45s
165:	learn: 0.1130845	total: 10.4s	remaining: 8.38s
166:	learn: 0.1121441	total: 10.4s	remaining: 8.31s
167:	learn: 0.1113050	total: 10.5s	remaining: 8.24s
168:	learn: 0.1102923	total: 10.5s	remaining: 8.17s
169:	learn: 0.1095124	total: 10.6s	remaining: 8.1s
170:	learn: 0.1084493	total: 10.7s	remaining: 8.04s
171:	learn: 0.1072233	total: 10.7s	remaining: 7.97s
172:	learn: 0.1067372	total: 10.8s	remaining: 7.9s
173:	learn: 0.1050588	total: 10.8s	remaining: 7.83s
174:	learn: 0.1041612	total: 10.9s	remaining: 7.76s
175:	learn: 0.1031151	total: 10.9s	remaining: 7.7s
176:	learn: 0.1021593	total: 11s	remaining: 7.63s
177:	learn: 0.1011924	total: 11s	remaining: 7.56s
178:	learn: 0.0999935	total: 11.1s	remaining: 7.49s
179:	learn: 0.0991738	total: 11.1s	remaining: 7.42s
180:	learn: 0.0985696	total: 11.2s	remaining: 7.36s
181:	learn: 0.09788

25:	learn: 0.3926143	total: 1.26s	remaining: 13.3s
26:	learn: 0.3891679	total: 1.31s	remaining: 13.3s
27:	learn: 0.3867494	total: 1.36s	remaining: 13.2s
28:	learn: 0.3832656	total: 1.4s	remaining: 13.1s
29:	learn: 0.3808236	total: 1.45s	remaining: 13.1s
30:	learn: 0.3748433	total: 1.5s	remaining: 13s
31:	learn: 0.3706486	total: 1.54s	remaining: 12.9s
32:	learn: 0.3656473	total: 1.59s	remaining: 12.9s
33:	learn: 0.3607216	total: 1.64s	remaining: 12.8s
34:	learn: 0.3579714	total: 1.69s	remaining: 12.8s
35:	learn: 0.3527547	total: 1.74s	remaining: 12.7s
36:	learn: 0.3503576	total: 1.78s	remaining: 12.7s
37:	learn: 0.3460474	total: 1.83s	remaining: 12.6s
38:	learn: 0.3426153	total: 1.88s	remaining: 12.6s
39:	learn: 0.3388984	total: 1.92s	remaining: 12.5s
40:	learn: 0.3354317	total: 1.97s	remaining: 12.5s
41:	learn: 0.3325430	total: 2.02s	remaining: 12.4s
42:	learn: 0.3286728	total: 2.07s	remaining: 12.3s
43:	learn: 0.3248223	total: 2.11s	remaining: 12.3s
44:	learn: 0.3210780	total: 2.16s	r

189:	learn: 0.0910241	total: 10.5s	remaining: 6.05s
190:	learn: 0.0900349	total: 10.5s	remaining: 6s
191:	learn: 0.0893873	total: 10.6s	remaining: 5.93s
192:	learn: 0.0886067	total: 10.6s	remaining: 5.88s
193:	learn: 0.0877596	total: 10.7s	remaining: 5.82s
194:	learn: 0.0867474	total: 10.7s	remaining: 5.77s
195:	learn: 0.0863315	total: 10.8s	remaining: 5.71s
196:	learn: 0.0854162	total: 10.8s	remaining: 5.65s
197:	learn: 0.0847993	total: 10.9s	remaining: 5.59s
198:	learn: 0.0841366	total: 10.9s	remaining: 5.53s
199:	learn: 0.0837590	total: 11s	remaining: 5.48s
200:	learn: 0.0829604	total: 11s	remaining: 5.42s
201:	learn: 0.0823406	total: 11.1s	remaining: 5.37s
202:	learn: 0.0814665	total: 11.1s	remaining: 5.31s
203:	learn: 0.0808745	total: 11.2s	remaining: 5.25s
204:	learn: 0.0804380	total: 11.2s	remaining: 5.19s
205:	learn: 0.0801069	total: 11.3s	remaining: 5.13s
206:	learn: 0.0795095	total: 11.3s	remaining: 5.08s
207:	learn: 0.0788520	total: 11.4s	remaining: 5.02s
208:	learn: 0.07826

49:	learn: 0.2792159	total: 2.46s	remaining: 12.3s
50:	learn: 0.2761459	total: 2.51s	remaining: 12.3s
51:	learn: 0.2742584	total: 2.56s	remaining: 12.2s
52:	learn: 0.2706085	total: 2.6s	remaining: 12.1s
53:	learn: 0.2663706	total: 2.65s	remaining: 12.1s
54:	learn: 0.2636661	total: 2.7s	remaining: 12s
55:	learn: 0.2611273	total: 2.75s	remaining: 12s
56:	learn: 0.2572221	total: 2.8s	remaining: 11.9s
57:	learn: 0.2557529	total: 2.84s	remaining: 11.9s
58:	learn: 0.2537368	total: 2.89s	remaining: 11.8s
59:	learn: 0.2501492	total: 2.94s	remaining: 11.8s
60:	learn: 0.2473961	total: 2.99s	remaining: 11.7s
61:	learn: 0.2457648	total: 3.03s	remaining: 11.6s
62:	learn: 0.2427191	total: 3.09s	remaining: 11.6s
63:	learn: 0.2384785	total: 3.13s	remaining: 11.6s
64:	learn: 0.2362362	total: 3.18s	remaining: 11.5s
65:	learn: 0.2330859	total: 3.23s	remaining: 11.5s
66:	learn: 0.2315398	total: 3.28s	remaining: 11.4s
67:	learn: 0.2287089	total: 3.33s	remaining: 11.4s
68:	learn: 0.2269378	total: 3.38s	rema

...

138:	learn: 0.1820809	total: 4.51s	remaining: 5.23s
139:	learn: 0.1805778	total: 4.55s	remaining: 5.2s
140:	learn: 0.1796682	total: 4.58s	remaining: 5.17s
141:	learn: 0.1781181	total: 4.61s	remaining: 5.13s
142:	learn: 0.1771788	total: 4.65s	remaining: 5.1s
143:	learn: 0.1753845	total: 4.68s	remaining: 5.07s
144:	learn: 0.1743597	total: 4.72s	remaining: 5.04s
145:	learn: 0.1729382	total: 4.75s	remaining: 5.01s
146:	learn: 0.1721784	total: 4.78s	remaining: 4.98s
147:	learn: 0.1708599	total: 4.82s	remaining: 4.95s
148:	learn: 0.1700269	total: 4.85s	remaining: 4.92s
149:	learn: 0.1691609	total: 4.89s	remaining: 4.89s
150:	learn: 0.1676359	total: 4.92s	remaining: 4.86s
151:	learn: 0.1666283	total: 4.95s	remaining: 4.82s
152:	learn: 0.1657569	total: 4.98s	remaining: 4.79s
153:	learn: 0.1642135	total: 5.02s	remaining: 4.76s
154:	learn: 0.1629897	total: 5.05s	remaining: 4.72s
155:	learn: 0.1620277	total: 5.08s	remaining: 4.69s
156:	learn: 0.1603334	total: 5.11s	remaining: 4.65s
157:	learn: 0.

298:	learn: 0.0663837	total: 9.62s	remaining: 32.2ms
299:	learn: 0.0659412	total: 9.65s	remaining: 0us
0:	learn: 0.6501055	total: 61.6ms	remaining: 18.4s
1:	learn: 0.6116817	total: 106ms	remaining: 15.9s
2:	learn: 0.5830862	total: 139ms	remaining: 13.8s
3:	learn: 0.5595516	total: 171ms	remaining: 12.7s
4:	learn: 0.5372882	total: 214ms	remaining: 12.6s
5:	learn: 0.5213808	total: 258ms	remaining: 12.6s
6:	learn: 0.5071489	total: 293ms	remaining: 12.3s
7:	learn: 0.4921703	total: 328ms	remaining: 12s
8:	learn: 0.4836297	total: 388ms	remaining: 12.5s
9:	learn: 0.4743571	total: 443ms	remaining: 12.8s
10:	learn: 0.4667791	total: 500ms	remaining: 13.1s
11:	learn: 0.4611208	total: 547ms	remaining: 13.1s
12:	learn: 0.4571463	total: 605ms	remaining: 13.4s
13:	learn: 0.4531746	total: 662ms	remaining: 13.5s
14:	learn: 0.4490752	total: 713ms	remaining: 13.5s
15:	learn: 0.4446801	total: 756ms	remaining: 13.4s
16:	learn: 0.4407513	total: 787ms	remaining: 13.1s
17:	learn: 0.4376059	total: 816ms	remaini

163:	learn: 0.1705388	total: 5.84s	remaining: 4.84s
164:	learn: 0.1692607	total: 5.88s	remaining: 4.81s
165:	learn: 0.1686549	total: 5.93s	remaining: 4.79s
166:	learn: 0.1674661	total: 5.99s	remaining: 4.77s
167:	learn: 0.1665184	total: 6.05s	remaining: 4.75s
168:	learn: 0.1654590	total: 6.1s	remaining: 4.73s
169:	learn: 0.1646322	total: 6.16s	remaining: 4.71s
170:	learn: 0.1628910	total: 6.22s	remaining: 4.69s
171:	learn: 0.1620160	total: 6.27s	remaining: 4.66s
172:	learn: 0.1611063	total: 6.32s	remaining: 4.64s
173:	learn: 0.1600017	total: 6.38s	remaining: 4.62s
174:	learn: 0.1587915	total: 6.44s	remaining: 4.6s
175:	learn: 0.1574789	total: 6.48s	remaining: 4.56s
176:	learn: 0.1565254	total: 6.5s	remaining: 4.52s
177:	learn: 0.1553613	total: 6.54s	remaining: 4.48s
178:	learn: 0.1543568	total: 6.57s	remaining: 4.44s
179:	learn: 0.1531292	total: 6.6s	remaining: 4.4s
180:	learn: 0.1518597	total: 6.63s	remaining: 4.36s
181:	learn: 0.1510128	total: 6.66s	remaining: 4.32s
182:	learn: 0.150

27:	learn: 0.4029609	total: 928ms	remaining: 9.01s
28:	learn: 0.4011128	total: 975ms	remaining: 9.12s
29:	learn: 0.3991545	total: 1.02s	remaining: 9.16s
30:	learn: 0.3964179	total: 1.05s	remaining: 9.13s
31:	learn: 0.3932465	total: 1.08s	remaining: 9.08s
32:	learn: 0.3906781	total: 1.12s	remaining: 9.04s
33:	learn: 0.3875640	total: 1.15s	remaining: 9.03s
34:	learn: 0.3848674	total: 1.19s	remaining: 9.04s
35:	learn: 0.3803716	total: 1.24s	remaining: 9.08s
36:	learn: 0.3757007	total: 1.28s	remaining: 9.08s
37:	learn: 0.3740696	total: 1.31s	remaining: 9.02s
38:	learn: 0.3718141	total: 1.34s	remaining: 8.96s
39:	learn: 0.3699243	total: 1.37s	remaining: 8.91s
40:	learn: 0.3674465	total: 1.41s	remaining: 8.9s
41:	learn: 0.3666856	total: 1.46s	remaining: 8.94s
42:	learn: 0.3651482	total: 1.5s	remaining: 8.96s
43:	learn: 0.3639167	total: 1.56s	remaining: 9.08s
44:	learn: 0.3617915	total: 1.62s	remaining: 9.16s
45:	learn: 0.3595914	total: 1.68s	remaining: 9.28s
46:	learn: 0.3573669	total: 1.74s

193:	learn: 0.1405196	total: 7.8s	remaining: 4.26s
194:	learn: 0.1397948	total: 7.83s	remaining: 4.22s
195:	learn: 0.1389840	total: 7.87s	remaining: 4.17s
196:	learn: 0.1380973	total: 7.9s	remaining: 4.13s
197:	learn: 0.1374922	total: 7.93s	remaining: 4.08s
198:	learn: 0.1366751	total: 7.96s	remaining: 4.04s
199:	learn: 0.1354557	total: 7.99s	remaining: 3.99s
200:	learn: 0.1346539	total: 8.02s	remaining: 3.95s
201:	learn: 0.1338761	total: 8.06s	remaining: 3.91s
202:	learn: 0.1330920	total: 8.1s	remaining: 3.87s
203:	learn: 0.1324798	total: 8.16s	remaining: 3.84s
204:	learn: 0.1317245	total: 8.2s	remaining: 3.8s
205:	learn: 0.1310199	total: 8.26s	remaining: 3.77s
206:	learn: 0.1305243	total: 8.31s	remaining: 3.73s
207:	learn: 0.1298728	total: 8.37s	remaining: 3.7s
208:	learn: 0.1293374	total: 8.4s	remaining: 3.66s
209:	learn: 0.1286850	total: 8.45s	remaining: 3.62s
210:	learn: 0.1276822	total: 8.49s	remaining: 3.58s
211:	learn: 0.1263874	total: 8.52s	remaining: 3.54s
212:	learn: 0.12578

--- Visualisation ---

In [25]:
precision_scores= {
                    'Tuned Decision Tree Precision:': precision_score(y_test, y_pred_tuned_dt, zero_division= 0),
                    'Tuned Decision Tree Precision1:': precision_score(y_test, y_pred_tuned_dt1, zero_division= 0),
                    'Tuned Decision Tree Precision2:': precision_score(y_test, y_pred_tuned_dt2, zero_division= 0),
                    'Tuned Random Forest Precision:': precision_score(y_test, y_pred_tuned_rf, zero_division= 0),
                    'Tuned Random Forest Precision1:': precision_score(y_test, y_pred_tuned_rf1, zero_division= 0),
                    'Tuned Random Forest Precision2:': precision_score(y_test, y_pred_tuned_rf2, zero_division= 0),
                    'Tuned AdaBoost Precision:': precision_score(y_test, y_pred_tuned_ada, zero_division= 0),
                    'Tuned AdaBoost Precision1:': precision_score(y_test, y_pred_tuned_ada1, zero_division= 0),
                    'Tuned AdaBoost Precision2:': precision_score(y_test, y_pred_tuned_ada2, zero_division= 0),
                    'Tuned AdaBoost Precision3:': precision_score(y_test, y_pred_tuned_ada3, zero_division= 0),
                    'GradientBoosting Precision:': precision_score(y_test, y_pred_grb, zero_division= 0),
                    'Tuned GradientBoosting Precision:': precision_score(y_test, y_pred_tuned_grb, zero_division= 0),
                    'Tuned GradientBoosting Precision1:': precision_score(y_test, y_pred_tuned_grb1, zero_division= 0),
                    'Tuned GradientBoosting Precision2:': precision_score(y_test, y_pred_tuned_grb2, zero_division= 0),
                    'Tuned CatBoost Precision:': precision_score(y_test, y_pred_tuned_cat, zero_division= 0),
                    'Tuned CatBoost Precision1:': precision_score(y_test, y_pred_tuned_cat1, zero_division= 0),
                    'Tuned CatBoost Precision2:': precision_score(y_test, y_pred_tuned_cat2, zero_division= 0),
                  }

rfe_precision= pd.DataFrame(list(precision_scores.items()), columns= ['Model', 'Precision Score'])
rfe_precision= rfe_precision.sort_values(by= 'Precision Score', ascending=False)
print(rfe_precision)

                                 Model  Precision Score
14           Tuned CatBoost Precision:         1.000000
10         GradientBoosting Precision:         0.411765
4      Tuned Random Forest Precision1:         0.402985
6            Tuned AdaBoost Precision:         0.400000
12  Tuned GradientBoosting Precision1:         0.390909
0       Tuned Decision Tree Precision:         0.388889
3       Tuned Random Forest Precision:         0.385827
8           Tuned AdaBoost Precision2:         0.384384
13  Tuned GradientBoosting Precision2:         0.383954
1      Tuned Decision Tree Precision1:         0.367713
9           Tuned AdaBoost Precision3:         0.362924
16          Tuned CatBoost Precision2:         0.360976
15          Tuned CatBoost Precision1:         0.356662
5      Tuned Random Forest Precision2:         0.353659
2      Tuned Decision Tree Precision2:         0.343558
11   Tuned GradientBoosting Precision:         0.325843
7           Tuned AdaBoost Precision1:         0

In [26]:
accuracy_scores= {
                    'Tuned Decision Tree Accuracy:': accuracy_score(y_test, y_pred_tuned_dt),
                    'Tuned Decision Tree Accuracy1:': accuracy_score(y_test, y_pred_tuned_dt1),
                    'Tuned Decision Tree Accuracy2:': accuracy_score(y_test, y_pred_tuned_dt2),
                    'Tuned Random Forest Accuracy:': accuracy_score(y_test, y_pred_tuned_rf),
                    'Tuned Random Forest Accuracy1:': accuracy_score(y_test, y_pred_tuned_rf1),
                    'Tuned Random Forest Accuracy2:': accuracy_score(y_test, y_pred_tuned_rf2),
                    'Tuned AdaBoost Accuracy:': accuracy_score(y_test, y_pred_tuned_ada),
                    'Tuned AdaBoost Accuracy1:': accuracy_score(y_test, y_pred_tuned_ada1),
                    'Tuned AdaBoost Accuracy2:': accuracy_score(y_test, y_pred_tuned_ada2),
                    'Tuned AdaBoost Accuracy3:': accuracy_score(y_test, y_pred_tuned_ada3),
                    'GradientBoosting Accuracy:': accuracy_score(y_test, y_pred_grb),                    
                    'Tuned GradientBoosting Accuracy:': accuracy_score(y_test, y_pred_tuned_grb),
                    'Tuned GradientBoosting Accuracy1:': accuracy_score(y_test, y_pred_tuned_grb1),
                    'Tuned GradientBoosting Accuracy2:': accuracy_score(y_test, y_pred_tuned_grb2),
                    'Tuned CatBoost Accuracy:': accuracy_score(y_test, y_pred_tuned_cat),
                    'Tuned CatBoost Accuracy1:': accuracy_score(y_test, y_pred_tuned_cat1),
                    'Tuned CatBoost Accuracy2:': accuracy_score(y_test, y_pred_tuned_cat2),
                  }

rfe_accuracy= pd.DataFrame(list(accuracy_scores.items()), columns= ['Model', 'Accuracy Score'])
rfe_accuracy= rfe_accuracy.sort_values(by= 'Accuracy Score', ascending=False)
print(rfe_accuracy)

                                Model  Accuracy Score
14           Tuned CatBoost Accuracy:        0.642327
7           Tuned AdaBoost Accuracy1:        0.640046
6            Tuned AdaBoost Accuracy:        0.638905
10         GradientBoosting Accuracy:        0.636623
11   Tuned GradientBoosting Accuracy:        0.624073
0       Tuned Decision Tree Accuracy:        0.621221
12  Tuned GradientBoosting Accuracy1:        0.614375
2      Tuned Decision Tree Accuracy2:        0.612664
4      Tuned Random Forest Accuracy1:        0.612094
3       Tuned Random Forest Accuracy:        0.608671
1      Tuned Decision Tree Accuracy1:        0.608100
8           Tuned AdaBoost Accuracy2:        0.597832
13  Tuned GradientBoosting Accuracy2:        0.595550
5      Tuned Random Forest Accuracy2:        0.586994
9           Tuned AdaBoost Accuracy3:        0.581860
15          Tuned CatBoost Accuracy1:        0.520251
16          Tuned CatBoost Accuracy2:        0.511694


In [27]:
auc_scores= {
                    'Tuned Decision Tree AUC:': roc_auc_score(y_test, y_pred_tuned_dt),
                    'Tuned Decision Tree AUC1:': roc_auc_score(y_test, y_pred_tuned_dt1),
                    'Tuned Decision Tree AUC2:': roc_auc_score(y_test, y_pred_tuned_dt2),
                    'Tuned Random Forest AUC:': roc_auc_score(y_test, y_pred_tuned_rf),
                    'Tuned Random Forest AUC1:': roc_auc_score(y_test, y_pred_tuned_rf1),
                    'Tuned Random Forest AUC2:': roc_auc_score(y_test, y_pred_tuned_rf2),
                    'Tuned AdaBoost AUC:': roc_auc_score(y_test, y_pred_tuned_ada),
                    'Tuned AdaBoost AUC1:': roc_auc_score(y_test, y_pred_tuned_ada1),
                    'Tuned AdaBoost AUC2:': roc_auc_score(y_test, y_pred_tuned_ada2),
                    'Tuned AdaBoost AUC3:': roc_auc_score(y_test, y_pred_tuned_ada3),
                    'GradientBoosting AUC:': roc_auc_score(y_test, y_pred_grb),
                    'Tuned GradientBoosting AUC:': roc_auc_score(y_test, y_pred_tuned_grb),
                    'Tuned GradientBoosting AUC1:': roc_auc_score(y_test, y_pred_tuned_grb1),
                    'Tuned GradientBoosting AUC2:': roc_auc_score(y_test, y_pred_tuned_grb2),
                    'Tuned CatBoost AUC:': roc_auc_score(y_test, y_pred_tuned_cat),
                    'Tuned CatBoost AUC1:': roc_auc_score(y_test, y_pred_tuned_cat1),
                    'Tuned CatBoost AUC2:': roc_auc_score(y_test, y_pred_tuned_cat2),
                  }

rfe_auc= pd.DataFrame(list(auc_scores.items()), columns= ['Model', 'AUC Score'])
rfe_auc= rfe_auc.sort_values(by= 'AUC Score', ascending=False)
print(rfe_auc)

                           Model  AUC Score
4      Tuned Random Forest AUC1:   0.514876
13  Tuned GradientBoosting AUC2:   0.511132
8           Tuned AdaBoost AUC2:   0.510800
12  Tuned GradientBoosting AUC1:   0.508916
3       Tuned Random Forest AUC:   0.508692
0       Tuned Decision Tree AUC:   0.506159
10         GradientBoosting AUC:   0.503386
16          Tuned CatBoost AUC2:   0.502780
1      Tuned Decision Tree AUC1:   0.502620
9           Tuned AdaBoost AUC3:   0.502224
6            Tuned AdaBoost AUC:   0.501295
14           Tuned CatBoost AUC:   0.500796
7           Tuned AdaBoost AUC1:   0.499370
15          Tuned CatBoost AUC1:   0.498543
5      Tuned Random Forest AUC2:   0.498134
2      Tuned Decision Tree AUC2:   0.497030
11   Tuned GradientBoosting AUC:   0.496423
